In [1]:
!pip install tensorflow==2.4.1
!pip install keras==2.4.3

In [2]:
from keras.layers import Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization, Reshape, Permute, Activation, Input, \
    add, multiply
from keras.layers import concatenate, core, Dropout
from keras.models import Model
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.optimizers import SGD
from keras.layers.core import Lambda
#import keras.backend as K

#%tensorflow_version 1.x
import os
import keras
from keras.callbacks import TensorBoard
import tensorflow as tf
#import keras.backend.tensorflow_backend as K
import keras.backend as K
import matplotlib.pyplot as plt
from keras.callbacks import CSVLogger


In [3]:
import os
import numpy as np
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split
import cv2

In [4]:
from __future__ import print_function
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
import os
import skimage.io as io
import skimage.transform as trans
import cv2
import warnings

warnings.filterwarnings("ignore")

BackGround = [255, 255, 255]
road = [0, 0, 0]
# COLOR_DICT = np.array([BackGround, road])
one = [128, 128, 128]
two = [128, 0, 0]
three = [192, 192, 128]
four = [255, 69, 0]
five = [128, 64, 128]
six = [60, 40, 222]
seven = [128, 128, 0]
eight = [192, 128, 128]
nine = [64, 64, 128]
ten = [64, 0, 128]
eleven = [64, 64, 0]
twelve = [0, 128, 192]
COLOR_DICT = np.array([one, two,three,four,five,six,seven,eight,nine,ten,eleven,twelve])


class data_preprocess:
    def __init__(self, train_path=None, image_folder=None, label_folder=None,
                 valid_path=None,valid_image_folder =None,valid_label_folder = None,
                 test_path=None, save_path=None,
                 img_rows=256, img_cols=256,
                 flag_multi_class=False,
                 num_classes = 2):
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.train_path = train_path
        self.image_folder = image_folder
        self.label_folder = label_folder
        self.valid_path = valid_path
        self.valid_image_folder = valid_image_folder
        self.valid_label_folder = valid_label_folder
        self.test_path = test_path
        self.save_path = save_path
        self.data_gen_args = dict(rotation_range=20,
                                  width_shift_range=0.002,
                                  shear_range=0.03,
                                  zoom_range=0.005,
                                  vertical_flip=True,
                                  horizontal_flip=True,
                                  fill_mode='nearest')
        self.image_color_mode = "rgb"
        self.label_color_mode = "grayscale"

        self.flag_multi_class = flag_multi_class
        self.num_class = num_classes
        self.target_size = (256, 256)
        self.img_type = 'png'

    def adjustData(self, img, label):
        if (self.flag_multi_class):
            img = img / 255.
            label = label[:, :, :, 0] if (len(label.shape) == 4) else label[:, :, 0]
            new_label = np.zeros(label.shape + (self.num_class,))
            for i in range(self.num_class):
                new_label[label == i, i] = 1
            label = new_label
        elif (np.max(img) > 1):
            #img = img / 255.
            #label = label / 255.
            #label[label >= 0.5] = 1
            #label[label < 0.5] = 0
            img2 =np.asarray(img)
            label2 =np.asarray(label)
            img2 =img2.astype('float32')
            label2 =label2.astype('float32')
            img2 /= 255.0
            label2 /= 255.0
            label2[label2 >= 0.5] = 1
            label2[label2 < 0.5] = 0
        return (img2, label2)

    def trainGenerator(self, batch_size, image_save_prefix="image", label_save_prefix="label",
                       save_to_dir=None, seed=7):
        '''
        can generate image and label at the same time
        use the same seed for image_datagen and label_datagen to ensure the transformation for image and label is the same
        if you want to visualize the results of generator, set save_to_dir = "your path"
        '''
        image_datagen = ImageDataGenerator(**self.data_gen_args)
        label_datagen = ImageDataGenerator(**self.data_gen_args)
        image_generator = image_datagen.flow_from_directory(
            self.train_path,
            classes=[self.image_folder],
            class_mode=None,
            color_mode=self.image_color_mode,
            target_size=self.target_size,
            batch_size=batch_size,
            save_to_dir=save_to_dir,
            save_prefix=image_save_prefix,
            seed=seed)
        label_generator = label_datagen.flow_from_directory(
            self.train_path,
            classes=[self.label_folder],
            class_mode=None,
            color_mode=self.label_color_mode,
            target_size=self.target_size,
            batch_size=batch_size,
            save_to_dir=save_to_dir,
            save_prefix=label_save_prefix,
            seed=seed)
        train_generator = zip(image_generator, label_generator)
        for (img, label) in train_generator:
            img, label = self.adjustData(img, label)
            yield (img, label)

    def testGenerator(self):
        filenames = os.listdir(self.test_path)
        for filename in filenames:
            img = io.imread(os.path.join(self.test_path, filename), as_gray=False)
            img = img / 255.
            img = trans.resize(img, self.target_size, mode='constant')
            img = np.reshape(img, img.shape + (1,)) if (not self.flag_multi_class) else img
            img = np.reshape(img, (1,) + img.shape)
            yield img

    def validLoad(self, batch_size,seed=7):
        image_datagen = ImageDataGenerator(rescale=0)
        label_datagen = ImageDataGenerator(rescale=0)
        image_generator = image_datagen.flow_from_directory(
            self.valid_path,
            classes=[self.valid_image_folder],
            class_mode=None,
            color_mode=self.image_color_mode,
            target_size=self.target_size,
            batch_size=batch_size,
            seed=seed)
        label_generator = label_datagen.flow_from_directory(
            self.valid_path,
            classes=[self.valid_label_folder],
            class_mode=None,
            color_mode=self.label_color_mode,
            target_size=self.target_size,
            batch_size=batch_size,
            seed=seed)
        train_generator = zip(image_generator, label_generator)
        for (img, label) in train_generator:
            img, label = self.adjustData(img, label)
            yield (img, label)
        # return imgs,labels

    def saveResult(self, npyfile, size, name,threshold=80):
        for i, item in enumerate(npyfile):
            img = item
            img_std = np.zeros((img.shape[0], img.shape[1], 3), dtype=np.uint8)
            if self.flag_multi_class:
                for row in range(len(img)):
                    for col in range(len(img[row])):
                        num = np.argmax(img[row][col])
                        img_std[row][col] = COLOR_DICT[num]
            else:
                for k in range(len(img)):
                    for j in range(len(img[k])):
                        num = img[k][j]
                        if num < (threshold/255.0):
                            img_std[k][j] = road
                        else:
                            img_std[k][j] = BackGround
            img_std = cv2.resize(img_std, size, interpolation=cv2.INTER_CUBIC)
            cv2.imwrite(os.path.join(self.save_path, ("%s_predict." + self.img_type) % (name)), img_std)

In [5]:
####  Metrics

from keras import backend as K
from keras.losses import binary_crossentropy
import tensorflow as tf
import numpy as np
def dice_coeff(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score
def dice_loss(y_true, y_pred):
    loss = 1 - dice_coeff(y_true, y_pred)
    return loss
def iou_coeff(y_true, y_pred):
    smooth=1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    union=K.sum(y_true_f) + K.sum(y_pred_f)-intersection
    mvalue=(intersection+smooth)/(union+smooth)
    return mvalue
def precision(y_true, y_pred):
    """Precision metric.

    Only computes a batch-wise average of precision.

    Computes the precision, a metric for multi-label classification of
    how many selected items are relevant.
    """
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision
def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall
def ACL5(y_true, y_pred): 

	#y_pred = K.cast(y_pred, dtype = 'float64')

	print(K.int_shape(y_pred))

	x = y_pred[:,1:,:,:] - y_pred[:,:-1,:,:] # horizontal and vertical directions 
	y = y_pred[:,:,1:,:] - y_pred[:,:,:-1,:]

	delta_x = x[:,1:,:-2,:]**2
	delta_y = y[:,:-2,1:,:]**2
	delta_u = K.abs(delta_x + delta_y) 

	epsilon = 0.00000001 # where is a parameter to avoid square root is zero in practice.
	w = 1####
	lenth = w * K.sum(K.sqrt(delta_u + epsilon)) # equ.(11) in the paper


	C_1 = np.ones((256, 256))
	C_2 = np.zeros((256, 256))

	region_in = K.abs(K.sum( y_pred[:,:,:,0] * ((y_true[:,:,:,0] - C_1)**2) ) ) # equ.(12) in the paper
	region_out = K.abs(K.sum( (1-y_pred[:,:,:,0]) * ((y_true[:,:,:,0] - C_2)**2) )) # equ.(12) in the paper

	lambdaP = 5 # lambda parameter could be various.
	
	loss =  lenth + lambdaP * ((region_in) + (region_out)) 

	return loss
def ACL5_mod(y_true, y_pred): 

	#y_pred = K.cast(y_pred, dtype = 'float64')

	print(K.int_shape(y_pred))

	x = y_pred[:,1:,:,:] - y_pred[:,:-1,:,:] # horizontal and vertical directions 
	y = y_pred[:,:,1:,:] - y_pred[:,:,:-1,:]

	delta_x = x[:,1:,:-2,:]**2
	delta_y = y[:,:-2,1:,:]**2
	delta_u = K.abs(delta_x + delta_y) 

	epsilon = 0.00000001 # where is a parameter to avoid square root is zero in practice.
	w = 1####
	lenth = w * K.sum(K.sqrt(delta_u + epsilon)) # equ.(11) in the paper


	C_1 = np.ones((256, 256))
	C_2 = np.zeros((256, 256))

	region_in = K.abs(K.sum( y_pred[:,:,:,0] * ((y_true[:,:,:,0] - C_1)**2) ) ) # equ.(12) in the paper
	region_out = K.abs(K.sum( (1-y_pred[:,:,:,0]) * ((y_true[:,:,:,0] - C_2)**2) )) # equ.(12) in the paper

	lambdaP = 5 # lambda parameter could be various.
	
	loss =  lenth + lambdaP * ((region_in) + (region_out*1.4)) 

	return loss

In [6]:
beta = 0.25
alpha = 0.25
gamma = 2
epsilon = 1e-5
smooth = 1

def tversky_index( y_true, y_pred):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (
                1 - alpha) * false_pos + smooth)

def tversky_loss( y_true, y_pred):
    return 1 - tversky_index(y_true, y_pred)

def focal_tversky( y_true, y_pred):
    pt_1 = tversky_index(y_true, y_pred)
    gamma = 0.75
    return K.pow((1 - pt_1), gamma)

def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

In [7]:
%env SM_FRAMEWORK=tf.keras

In [8]:
!pip install segmentation_models
import segmentation_models
from segmentation_models.losses import bce_jaccard_loss

In [9]:
def ACL5_bce_jaccard_loss(y_true, y_pred):
    loss = ACL5(y_true, y_pred) + bce_jaccard_loss(y_true, y_pred)
    return loss


def focal_tversky_bce_jaccard_loss(y_true, y_pred):
    loss = focal_tversky(y_true, y_pred) + 2*bce_jaccard_loss(y_true, y_pred)
    return loss


In [10]:

from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Conv2D , BatchNormalization , Activation , MaxPool2D , Input , Dropout , ZeroPadding2D , Conv2DTranspose , Concatenate
from tensorflow.keras.applications import DenseNet201

from tensorflow.keras.models import Model

from tensorflow.keras.applications import InceptionResNetV2

In [11]:
from keras.layers import Input, Conv2D, UpSampling2D, BatchNormalization, Activation, add,average,concatenate , Conv2DTranspose
from keras.layers.core import Lambda
from keras.optimizers import *
from keras.losses import binary_crossentropy
#import tensorflow as tf
#import keras.backend as K

from keras.layers import Lambda, SeparableConv2D

IMG_SIZE = 256
h_heuns_method=0.5

def res_block(x, nb_filters, strides):
    res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same' ,strides=strides[0])(x)
    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    
    res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same',strides=strides[1])(res_path)
    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    hpath = Lambda(lambda x: x * h_heuns_method)(res_path)

    shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1),strides=strides[0])(x)
    shortcut = BatchNormalization()(shortcut)
    shortcut = Activation(activation='relu')(shortcut)

    res_path = add([shortcut, hpath])#suma corta
    return res_path
def res_block2(x,y,nb_filters, strides):
    
    
    res_path = Conv2D(filters=nb_filters[0], kernel_size=(3, 3), padding='same' ,strides=strides[0])(x)
    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    
    res_path = Conv2D(filters=nb_filters[1], kernel_size=(3, 3), padding='same',strides=strides[1])(res_path)
    res_path = BatchNormalization()(res_path)
    res_path = Activation(activation='relu')(res_path)
    hpath = Lambda(lambda x: x * h_heuns_method)(res_path)

    shortcut = Conv2D(nb_filters[1], kernel_size=(1, 1),strides=strides[0])(x)
    shortcut = BatchNormalization()(shortcut)
    shortcut = Activation(activation='relu')(shortcut)
    
    res_path = add([shortcut, hpath])#suma corta
    res_path = BatchNormalization()(res_path)  
    res_path = Activation(activation='relu')(res_path)
    # I think it will come , as we want to make y and res_path having SIMILIAR VALUES
    
    res_path = average([y, res_path])#suma doble 
    return res_path


def encoder(x):
    to_decoder = []

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1 , 1))(x)
    main_path = BatchNormalization()(main_path)
    main_path = Activation(activation='relu')(main_path)

    main_path = Conv2D(filters=64, kernel_size=(3, 3), padding='same', strides=(1, 1) )(main_path)
    main_path = BatchNormalization()(main_path)
    main_path = Activation(activation='relu')(main_path)
    hpath = Lambda(lambda x: x * h_heuns_method)(main_path)

    shortcut = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1)  )(x)
    shortcut = BatchNormalization()(shortcut)
    main_path = Activation(activation='relu')(main_path)
    main_path = add([shortcut, hpath])#suma corta

    to_decoder.append(main_path)


    s1 = Conv2D(filters=128, kernel_size=(1, 1), strides=(2, 2) )(x)
    s1 = BatchNormalization()(s1)
    s1 = Activation(activation='relu')(s1)
    
    main_path = res_block2(main_path,s1, [128, 128], [(2, 2), (1, 1)]) 
    to_decoder.append(main_path)

    main_path = res_block(main_path, [256, 256], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    s2 = Conv2D(filters=512, kernel_size=(1, 1), strides=(4, 4) )(to_decoder[1])
    s2 = BatchNormalization()(s2)
    s2 = Activation(activation='relu')(s2)
    
    main_path = res_block2(main_path,s2, [512, 512], [(2, 2), (1, 1)])
    to_decoder.append(main_path)

    return to_decoder


def decoder(x, from_encoder):
    
    main_path = UpSampling2D(size=(2, 2))(x)#32x32
    #main_path = Conv2DTranspose(512, [2, 2], strides=[2, 2])(x)
    main_path1 = concatenate([main_path, from_encoder[3]], axis=3)
    main_path = res_block(main_path1, [512, 512], [(1, 1), (1, 1)])

    main_path = UpSampling2D(size=(2, 2))(main_path)###64x64
    #main_path = Conv2DTranspose(512, [2, 2], strides=[2, 2])(main_path)
    main_path = concatenate([main_path, from_encoder[2]], axis=3)#
    u1 = UpSampling2D(size=(2, 2))(main_path1)#
    #u1 = Conv2DTranspose(256, [2, 2], strides=[2, 2])(main_path1)
    u1 = Conv2D(256, kernel_size=(1, 1),strides=(1, 1))(u1)
    u1 = BatchNormalization()(u1)
    main_path = res_block2(main_path,u1, [256, 256], [(1, 1), (1, 1)])#

    main_path = UpSampling2D(size=(2, 2))(main_path)#128x128
    #main_path = Conv2DTranspose(128, [2, 2], strides=[2, 2])(main_path)
    main_path2 = concatenate([main_path, from_encoder[1]], axis=3)#
    main_path = res_block(main_path2, [128, 128], [(1, 1), (1, 1)])#

    main_path = UpSampling2D(size=(2, 2))(main_path)#256x256
    #main_path = Conv2DTranspose(64, [2, 2], strides=[2, 2])(main_path)
    main_path = concatenate([main_path, from_encoder[0]], axis=3)#256x256

    u2 = UpSampling2D(size=(2,2))(main_path2)#
    #u2 = Conv2DTranspose(64, [2, 2], strides=[2, 2])(main_path2)
    u2 = Conv2D(64, kernel_size=(1, 1),strides=(1, 1))(u2)#
    u2 = BatchNormalization()(u2)
    main_path = res_block2(main_path,u2, [64, 64], [(1, 1), (1, 1)])#256x256

    return main_path


def res2unet(lrate=7.00E-05,pretrained_weights=None):
    print(lrate)
    input_size=(IMG_SIZE, IMG_SIZE, 3)
    inputs = Input(shape=input_size)

    to_decoder = encoder(inputs)

### Bridge
    #path = res_block(to_decoder[3], [1024, 1024], [(2, 2), (1, 1)])####bridge
    path = SeparableConv2D(1024, (3,3), activation = 'relu', padding = 'same')(to_decoder[-1])
    path = SeparableConv2D(1024, (3,3), activation = 'relu', padding = 'same')(path)
    path = BatchNormalization()(path)
    path = MaxPooling2D(pool_size = (2,2))(path)
    path = Dropout(0.2)(path)
    
    
###  DECODER
    path = decoder(path, from_encoder=to_decoder)


    path = Conv2D(2, kernel_size=(3, 3),activation='relu', padding='same', strides=(1, 1))(path)
    path = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(path)
    model = Model(inputs=inputs, outputs=path)
    model.compile(optimizer=Adam(lr=lrate), loss=bce_jaccard_loss, metrics=[ACL5 ,bce_jaccard_loss ,  dice_loss,iou_coeff,precision,recall])
    model.summary()
    if (pretrained_weights):
        model.load_weights(pretrained_weights)
    return model

In [12]:
# Even the skip features were extracted for the Decoder purposes to feed in the required Skip Features


from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import InceptionResNetV2

from glob import glob
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Conv2D , BatchNormalization , Activation , MaxPool2D , Input , Dropout , ZeroPadding2D , Conv2DTranspose , Concatenate

import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import tensorflow as tf
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply, AveragePooling2D, UpSampling2D
from tensorflow.keras.models import Model
from tensorflow.keras.applications import VGG19
from tensorflow.keras.applications import InceptionResNetV2

from glob import glob
from sklearn.model_selection import train_test_split
import cv2
from tensorflow.keras.applications import EfficientNetB7
from tensorflow.keras.layers import Conv2D , BatchNormalization , Activation , MaxPool2D , Input , Dropout , ZeroPadding2D , Conv2DTranspose , Concatenate


def squeeze_excite_block(inputs, ratio=8):
    init = inputs       ## (b, 128, 128, 32)
    channel_axis = -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)     ## (b, 32)   -> (b, 1, 1, 32)
    se = Reshape(se_shape)(se)
    se = Dense(filters//ratio, activation="relu", use_bias=False)(se)
    se = Dense(filters, activation="sigmoid", use_bias=False)(se)

    x = Multiply()([inputs, se])
    return x

def ASPP(x, filter):
    shape = x.shape

    y1 = AveragePooling2D(pool_size=(shape[1], shape[2]))(x)
    y1 = Conv2D(filter, 1, padding="same")(y1)
    y1 = BatchNormalization()(y1)
    y1 = Activation("relu")(y1)
    y1 = UpSampling2D((shape[1], shape[2]), interpolation="bilinear")(y1)
    y1 = Dropout(0.3)(y1)
    
    y2 = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(x)
    y2 = BatchNormalization()(y2)
    y2 = Activation("relu")(y2)
    
    y2 = Dropout(0.3)(y2)
    y3 = Conv2D(filter, 3, dilation_rate=6, padding="same", use_bias=False)(x)
    y3 = BatchNormalization()(y3)
    y3 = Activation("relu")(y3)
    y3 = Dropout(0.3)(y3)
    
    y4 = Conv2D(filter, 3, dilation_rate=12, padding="same", use_bias=False)(x)
    y4 = BatchNormalization()(y4)
    y4 = Activation("relu")(y4)
    y4 = Dropout(0.3)(y4)
    
    y5 = Conv2D(filter, 3, dilation_rate=18, padding="same", use_bias=False)(x)
    y5 = BatchNormalization()(y5)
    y5 = Activation("relu")(y5)
    y5 = Dropout(0.3)(y5)
    
    y = Concatenate()([y1, y2, y3, y4, y5])

    y = Conv2D(filter, 1, dilation_rate=1, padding="same", use_bias=False)(y)
    y = BatchNormalization()(y)
    y = Activation("relu")(y)
    y= Dropout(0.2)(y)
    
    return y

def conv_block(x, filters):
    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)
    
    x = Conv2D(filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    x = Dropout(0.3)(x)
    x = squeeze_excite_block(x)

    return x

def encoder1(inputs):
    skip_connections = []
    model = InceptionResNetV2(include_top = False  , weights = "imagenet" , input_tensor = inputs)
    #Skip Connection
    s1 = model.get_layer("input_1").output                 #Shape will be 256 * 256

    s2 = model.get_layer("activation_2").output         # 125 * 125 
    s2 = ZeroPadding2D( ((2 , 1) ,  ( 2 , 1)) )(s2)          # 128 * 128


    s3 = model.get_layer("activation_4").output        # 60 * 60
    s3 = ZeroPadding2D( ((2 , 2 ) , (2, 2)) )(s3)         # 64 * 64 


    s4 = model.get_layer("activation_74").output                              # 29*29
    s4 = ZeroPadding2D( ((2 , 1) ,  ( 2 , 1)) )(s4) 
                                                                                # 32 * 32
    # Padded with 2 on top , 1 in bottom     2 in left and 1 in right


    b1 = model.get_layer("activation_161").output                      # 14 * 14

    #model = VGG19(include_top=False, weights="imagenet", input_tensor=inputs)
    names = ["input_1", "activation_2", "activation_4", "activation_74"]

    for feature in [s1 , s2 , s3 , s4]:

        skip_connections.append(feature)


    output = model.get_layer("activation_161").output
    output = ZeroPadding2D( ((1 , 1) , (1, 1)) )(output)                         # 16 * 16 

    return output, skip_connections

def decoder1(inputs, skip_connections):
    num_filters = [512,256, 128, 64]
    skip_connections.reverse()

    x = inputs
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation="bilinear")(x)
        x = Concatenate()([x, skip_connections[i]])
        x = conv_block(x, f)

    return x

def output_block(inputs):
    x = Conv2D(2, 3, padding="same")(inputs)
    x = Conv2D(1, 1, padding="same")(inputs)
    x = Activation("sigmoid")(x)
    return x

def encoder2(inputs):
    num_filters = [ 64, 128, 256 , 512]
    skip_connections = []

    x = inputs

    for i, f in enumerate(num_filters):
        x = conv_block(x, f)
        skip_connections.append(x)
        x = MaxPool2D((2, 2))(x)

    return x, skip_connections

def decoder2(inputs, skip_1, skip_2):
    num_filters = [512 , 256, 128, 64]
    skip_2.reverse()

    x = inputs
    for i, f in enumerate(num_filters):
        x = UpSampling2D((2, 2), interpolation="bilinear")(x)
        x = Concatenate()([x, skip_1[i], skip_2[i]])
        x = conv_block(x, f)

    return x



    
def build_model(input_shape , pretrained_weights = None):
    
    
    inputs = Input(input_shape)
    x, skip_1 = encoder1(inputs)
    x = ASPP(x, 64)
    x = decoder1(x, skip_1)
    output1 = output_block(x)

    x1 = inputs * output1
    
    
    ## 2nd Unet
    to_decoder = encoder(x1)

### Bridge
    #path = res_block(to_decoder[3], [1024, 1024], [(2, 2), (1, 1)])####bridge
    path = SeparableConv2D(1024, (3,3), activation = 'relu', padding = 'same')(to_decoder[-1])
    path = SeparableConv2D(1024, (3,3), activation = 'relu', padding = 'same')(path)
    path = BatchNormalization()(path)
    path = MaxPooling2D(pool_size = (2,2))(path)
    path = Dropout(0.2)(path)
    
    
###  DECODER
    path = decoder(path, from_encoder=to_decoder)
    
    path = Conv2D(2, kernel_size=(3, 3),activation='relu', padding='same', strides=(1, 1))(path)
    path = Conv2D(filters=1, kernel_size=(1, 1), activation='sigmoid')(path)
    model = Model(inputs=inputs, outputs=path)

    model.compile(optimizer=Adam(lr=7.00E-05), loss=bce_jaccard_loss, metrics=[ACL5 ,bce_jaccard_loss ,  dice_loss,iou_coeff,precision,recall])

    model.summary()
    if (pretrained_weights):
        model.load_weights(pretrained_weights)
    return model
    
    

    
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


#path to images
train_path = "../input/training/training"
image_folder = "images"
label_folder = "label"
valid_path =  "../input/validation/Validation"
valid_image_folder ="images"
valid_label_folder = "label"
log_filepath = './log'
flag_multi_class = False
num_classes = 2
dp = data_preprocess(train_path=train_path,image_folder=image_folder,label_folder=label_folder,
                     valid_path=valid_path,valid_image_folder=valid_image_folder,valid_label_folder=valid_label_folder,
                     flag_multi_class=flag_multi_class,
                     num_classes=num_classes)

train_data = dp.trainGenerator(batch_size=2)
valid_data = dp.validLoad(batch_size=1)
test_data = dp.testGenerator()
#model = res2unet(lrate=7.00E-05 )
                 #pretrained_weights = "../input/Weights/Res2Net_Conv2DTranspose (2).hdf5" )

from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.layers import GlobalAveragePooling2D, Reshape, Dense, Multiply, AveragePooling2D, UpSampling2D
if __name__ == "__main__":
    
    model = build_model(input_shape = ( 256 , 256 , 3) )
    model.summary()
    model_checkpoint1 = keras.callbacks.ModelCheckpoint('DoubleUnet.hdf5', monitor='val_dice_loss',verbose=1,mode='min',save_best_only=True)
    csv_logger = CSVLogger('trainingRes2Net.log', append=True, separator=';')



In [13]:
!pip install patchify
model.load_weights("../input/doubleunet-res2net-bcejaccard-weights/Double_Res2UNet_BCEJaccard.hdf5")
from keras.utils import normalize
from matplotlib import pyplot as plt
from patchify import patchify, unpatchify
def read_image(path):
    x = io.imread(path, as_gray=False)
    x = x / 255.0

    return x

test_path = '../input/imagestest/Images_to_Test/2'
test_image_folder ="images"
Test_path = test_path + '/' + test_image_folder
test_label_folder ="labels"


import os
  
# Directory
directory = "RESULT"
  
# Parent Directory path
parent_dir = "./"
  
# Path
path = os.path.join(parent_dir, directory)
os.mkdir(path)


from tqdm import tqdm

def read_image(path):
    x = io.imread(path, as_gray=False)
    x = x / 255.0
    
    x = trans.resize(x, (256, 256), mode='constant')

    return x

def read_mask(path):
    #x = io.imread(path, as_gray=False)
    #x = x / 255.0
    #x = trans.resize(x, (256, 256), mode='constant')
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    x = cv2.resize(x, ( 256 , 256))
    x = np.expand_dims(x, axis=-1)
    return x



def mask_parse(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask


def tf_parse(imagepath , maskpath):
  def _parse(imagepath , maskpath):
    x = read_image(imagepath)
    y = read_mask(maskpath)

    return x , y

  x , y = tf.numpy_function(_parse , [imagepath , maskpath] , [tf.float64 , tf.float64] )
  x.set_shape([256 , 256 , 3])
  y.set_shape([256, 256, 1])

  return x , y 


def tf_dataset( imagepath , maskpath , batch = 2):
  dataset = tf.data.Dataset.from_tensor_slices((imagepath , maskpath))
  dataset = dataset.map(tf_parse)
  dataset = dataset.batch(batch)
  dataset = dataset.repeat()
  return dataset


if __name__ == "__main__":
    ## Dataset
    
    batch_size = 8
    #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data("/content/drive/MyDrive/Test")
    test_x = os.listdir(os.path.join(test_path ,test_image_folder ))
    test_y = os.listdir(os.path.join(test_path ,test_label_folder ))
    
    TEST_X = []
    TEST_Y = []
    for img_file_name in test_x:
        TEST_X.append('../input/imagestest/Images_to_Test/2/images' + "/" +img_file_name)
        
    for label_file_name in test_y:
        TEST_Y.append('../input/imagestest/Images_to_Test/2/labels' + "/" +label_file_name)
    test_dataset = tf_dataset( TEST_X , TEST_Y, batch=batch_size)

    test_steps = (len(TEST_X)//batch_size)
    if len(TEST_X) % batch_size != 0:
        test_steps += 1

    #with CustomObjectScope({'iou': iou}):
    #    model = tf.keras.models.load_model("/content/drive/model.h5")

    #model.evaluate(test_dataset, steps=test_steps)
    for i, (x, y) in tqdm(enumerate(zip(TEST_X, TEST_Y)), total=len(TEST_X)):
        name = x.split(".")[2].split("/")[-1]
        x = read_image(x)
        y = read_mask(y)
        #model.load_weights("./Double_Res2UNet.hdf5")
        y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0

        all_images = [
            x * 255.0, white_line,
            mask_parse(y), white_line,
            mask_parse(y_pred) * 255.0
        ]
        image = np.concatenate(all_images, axis=1)
        
        cv2.imwrite(f"./RESULT/{name}.png", image)
  
test_path = '../input/imagestest/Images_to_Test/1'
from tqdm import tqdm
  

if __name__ == "__main__":
    ## Dataset
    
    batch_size = 8
    #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data("/content/drive/MyDrive/Test")
    test_x = os.listdir(os.path.join(test_path ,test_image_folder ))
    test_y = os.listdir(os.path.join(test_path ,test_label_folder ))
    
    TEST_X = []
    TEST_Y = []
    for img_file_name in test_x:
        TEST_X.append('../input/imagestest/Images_to_Test/1/images' + "/" +img_file_name)
        
    for label_file_name in test_y:
        TEST_Y.append('../input/imagestest/Images_to_Test/1/labels' + "/" +label_file_name)
    test_dataset = tf_dataset( TEST_X , TEST_Y, batch=batch_size)

    test_steps = (len(TEST_X)//batch_size)
    if len(TEST_X) % batch_size != 0:
        test_steps += 1

    for i, (x, y) in tqdm(enumerate(zip(TEST_X, TEST_Y)), total=len(TEST_X)):
        name = x.split(".")[2].split("/")[-1]
        x = read_image(x)
        y = read_mask(y)
        #model.load_weights("./Double_Res2UNet.hdf5")
        y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0

        all_images = [
            x * 255.0, white_line,
            mask_parse(y), white_line,
            mask_parse(y_pred) * 255.0
        ]
        image = np.concatenate(all_images, axis=1)
        
        cv2.imwrite(f"./RESULT/{name}.png", image)

os.chdir(r'/kaggle/working')

from zipfile import ZipFile
import os
from os.path import basename
dirName = "./RESULT"
# create a ZipFile object
with ZipFile('sampleDir.zip', 'w') as zipObj:
   # Iterate over all the files in directory
   for folderName, subfolders, filenames in os.walk(dirName):
       for filename in filenames:
           #create complete filepath of file in directory
           filePath = os.path.join(folderName, filename)
           # Add file to zip
           zipObj.write(filePath, basename(filePath))


test_path = '../input/testdata/Test'
test_image_folder ="images"
Test_path = test_path + '/' + 'test_image_folder'
test_label_folder ="label"


import os
  
# Directory
directory = "RESULT_Test"
  
# Parent Directory path
parent_dir = "./"
  
# Path
path = os.path.join(parent_dir, directory)
os.mkdir(path)


from tqdm import tqdm

if __name__ == "__main__":
    ## Dataset
    
    batch_size = 8
    #(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data("/content/drive/MyDrive/Test")
    test_x = os.listdir(os.path.join(test_path ,test_image_folder ))
    test_y = os.listdir(os.path.join(test_path ,test_label_folder ))
    
    TEST_X = []
    TEST_Y = []
    for img_file_name in test_x:
        TEST_X.append('../input/testdata/Test/images' + "/" +img_file_name)
        
    for label_file_name in test_y:
        TEST_Y.append('../input/testdata/Test/label'+ "/" +label_file_name)
    test_dataset = tf_dataset( TEST_X , TEST_Y, batch=batch_size)

    test_steps = (len(TEST_X)//batch_size)
    if len(TEST_X) % batch_size != 0:
        test_steps += 1

    #with CustomObjectScope({'iou': iou}):
    #    model = tf.keras.models.load_model("/content/drive/model.h5")

    #model.evaluate(test_dataset, steps=test_steps)
    
    for i, (x, y) in tqdm(enumerate(zip(TEST_X, TEST_Y)), total=len(TEST_X)):
        name = x.split(".")[2].split("/")[-1]
        if read_image(x).shape[-1] == 4:

          x = read_image(x)[:,:,:-1]
        else:
          x = read_image(x)

        y = read_mask(y)
        y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
        h, w, _ = x.shape
        white_line = np.ones((h, 10, 3)) * 255.0

        all_images = [
            x * 255.0, white_line,
            mask_parse(y), white_line,
            mask_parse(y_pred) * 255.0
        ]
        image = np.concatenate(all_images, axis=1)
        
        cv2.imwrite(f"./RESULT_Test/{name}.png", image)

iou_score_list = []

precision_score_list = []
recall_score_list = []
dice_coeff_score_list = []

for i, (x, y) in tqdm(enumerate(zip(TEST_X, TEST_Y)), total=len(TEST_X)):
    if read_image(x).shape[-1] == 4:

      x = read_image(x)[:,:,:-1]
    else:
      x = read_image(x)
    y = read_mask(y)
    
    #y_pred = model.predict(np.expand_dims(x, axis=0))
    y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
    y_pred = np.reshape(y_pred , -1)
    y_pred = np.array([ 1.0 if values else 0.0 for values in y_pred])
    y = y > 0.5
    y = np.reshape(y , -1)
    y = np.array([ 1.0 if values else 0.0 for values in y])
    y_pred = np.reshape(y_pred , (256 ,256 , 1))
    y = np.reshape(y , (256 ,256 , 1))
    #y = y.astype("float32")
    #y_pred = y_pred.astype("float32")
    
    
    iou_score_list.append(iou_coeff(y , y_pred))
    precision_score_list.append(precision(y , y_pred))
    recall_score_list.append(recall(y , y_pred))
    dice_coeff_score_list.append(dice_coeff(y , y_pred))

print("Average_IOU :", np.mean(iou_score_list)) 
print("Average_Precision :", np.mean(precision_score_list)) 
print("Average_Recall :", np.mean(recall_score_list)) 
print("Average_Dice_Coeff :", np.mean(dice_coeff_score_list)) 

import shutil

shutil.make_archive("RESULT_Test", 'zip', "./RESULT_Test")


In [14]:
os.chdir(r'/kaggle/working')
from IPython.display import FileLink

FileLink(r'./RESULT_Test.zip')


In [15]:
patch_size=256
from PIL import Image
large_image = io.imread('../input/imagestest/Images_to_Test/1/images/i1_.png' , as_gray = False )
large_image = large_image/ 255.0
large_image = trans.resize(large_image, (2560, 2560), mode='constant')

patches = patchify(large_image, ( 256 , 256 , 3), step= 256 )  #Step=256 for 256 patches means no overlap

predicted_patches = []
for i in range(patches.shape[0]):
    for j in range(patches.shape[1]):
        #print(i,j)
        
        single_patch = patches[i,j,:,: ,:]   # (1 , 256 , 256 , 3)
        
        
        #single_patch = single_patch/255.0
        #single_patch_norm = np.expand_dims(np.array(single_patch), axis=1)
        single_patch=np.expand_dims(single_patch, 0)
        #print(single_patch.shape)
        
#Predict and threshold for values above 0.5 probability
        single_patch_prediction = (model.predict(single_patch[0,:,:,:])[0] > 0.5)
        single_patch_prediction = single_patch_prediction * 255.0
        predicted_patches.append(single_patch_prediction)

predicted_patches = np.array(predicted_patches)


predicted_patches_reshaped = np.reshape(predicted_patches, (patches.shape[0], patches.shape[1], 256,256) )
reconstructed_image = unpatchify(predicted_patches_reshaped, large_image.shape[:2])
plt.imshow(reconstructed_image, cmap='gray')
plt.imsave('./segm1.png', reconstructed_image, cmap='gray')

plt.hist(reconstructed_image.flatten())  #Threshold everything above 0

final_prediction = (reconstructed_image > 0.01).astype(np.uint8)

plt.figure(figsize=(8, 8))
plt.subplot(221)
plt.title('Large Image')
plt.imshow(large_image, cmap='gray')
plt.subplot(222)
plt.title('Prediction of large Image')
plt.imshow(reconstructed_image, cmap='gray')
plt.subplot(223)
plt.title('final_prediction')
plt.imshow(final_prediction, cmap='gray')
plt.show()